<center><h1>🔥  FHIR Kindling</h1></center>

Python fhir client library for easier and safer interactions with FHIR servers and resources


## Features

- Create, Read, Update and Delete resources using a server's REST API
- Resource validation
- Transfer resources between fhir servers
- CSV/Dataframe serialization for resources
- Synthetic data generation and upload


## Installation

Install the latest published version from pypi:
```bash
pip install --user fhir-kindling
```
or install the newest version directly from github:
```bash
pip install --user git+https://github.com/migraf/fhir-kindling.git
```


In [ ]:
!pip install --upgrade fhir-kindling

<center><h2>👨‍💻   How to use the library</h2></center>

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from fhir_kindling import FhirServer

_ = load_dotenv(find_dotenv())

In [ ]:

fhir_api = "https://demo.personalhealthtrain.de/demo-fhir-3"
username = os.getenv("DEMO_USER")
password = os.getenv("DEMO_PW")
server = FhirServer(
    api_address=fhir_api,
    username=username,
    password=password,
)

## Query for resources

Query the server with the `query()` method of the server class.

There are three ways to define a query:
- Iteratively build the query on a resource using methods like `where()`, `include()`, `has()`
- Use a `query_string` to define the query ie `Patient?_id=123"`
- Pass a `FHIRQueryParameters` object to the query method

## Iteratively building a query

Start building a query by selecting the base resource first

In [ ]:
query = server.query("Patient")
query.query_url

### Querying the server
the query is executed against the server using one of the methods `all()`, `first()`, `limit()`

In [ ]:
response = query.all()
response

In [ ]:
response = query.limit(5)
response

Accessing the resources in a `QueryResponse` object.

In [ ]:
response.resources[0]

### Adding filter conditions

Filter parameters are added on the fields of the base resource using the `where()` method.

In [ ]:
query_2 = server.query("Patient").where("birthdate", "lt", "2000-01-01")
query_2.query_url

In [ ]:
query_2.all()

### Including related resources

In [ ]:
query_3 = query_2.include(resource="Condition", reference_param="subject", reverse=True)
query_3.query_url

In [ ]:
resp = query_3.all()
resp

## Working with the response

The response to the query is a `QueryResponse` object.

- The `resources` attribute contains a list of resources of the base resource type returned by the query
- The `included_resources` attribute contains a list of included resources. Each entry in the


In [ ]:
[resource.resource_type for resource in resp.included_resources]

In [ ]:
resp.included_resources[0].resources[0]

### Saving the response

Responses can be saved to a file using the `save()` method of the `QueryResponse` class.
Supported formats are `json`, `xml` (if the query was executed with `xml` format) and `csv`.

In [ ]:
path = os.path.join(os.getcwd(), "query_response.json")
resp.save(file_path=path)

In [ ]:
with open(path, "r") as f:
    print("".join(f.readlines()[:8]))

### Serializing resources into a pandas dataframe

A response (or any bundle) can be serialized into pandas dataframes.
If the response contains resources of different types, the resources are serialized into separate dataframes for each type.

In [ ]:
dfs = resp.to_dfs()

In [ ]:
dfs[0].head()

In [ ]:
dfs[1].head()

### Converting a list of resources to a dataframe

Any list of resources (pydantic models or dicts) can be converted to a dataframe using the `flattten()` method.

In [ ]:
from fhir_kindling.serde import flatten_resources

# get a list of patient resources
patients = server.query("Patient").limit(100).resources

In [ ]:
flatten_resources(patients)

## Generating synthetic data

Generate complex synthetic data sets using dataset and resource generator functions.
Interdependencies between resources and the likelihood of a resource being generated can be defined.


This example will generate a dataset with:
- Patients
- with Covid-19 conditions
- a certain likelihood of being vaccinated.

Start by importing and defining some constants i.e. Codes for the condition and the vaccination.

In [ ]:
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

from fhir_kindling.generators.patient import PatientGenerator
from fhir_kindling.generators.resource_generator import ResourceGenerator, GeneratorParameters, FieldValue
from fhir_kindling.generators.field_generator import FieldGenerator
from fhir_kindling.generators.dataset import DatasetGenerator
from fhir_kindling.fhir_query.query_parameters import QueryOperators

import pendulum

covid_code = CodeableConcept(
    coding=[
        Coding(
            system="http://id.who.int/icd/release/11/mms",
            code="RA01.0",
            display="COVID-19, virus identified"
        )
    ],
    text="COVID-19"
)

vaccination_code = CodeableConcept(
    coding=[
        Coding(
            system="http://id.who.int/icd/release/11/mms",
            code="XM0GQ8",
            display="COVID-19 vaccine, RNA based"
        )
    ],
    text="COVID vaccination"
)


Configure the data set generator and subgenerators

In [ ]:
count = 100
dataset_generator = DatasetGenerator("Patient", n=count)

covid_params = GeneratorParameters(
    field_values=[
        FieldValue(field="code", value=covid_code),
    ]
)

covid_generator = ResourceGenerator("Condition", generator_parameters=covid_params)
# add covid conditions to patients
dataset_generator.add_resource(covid_generator, name="covid")

# patients, patient_ids = PatientGenerator(n=count, generate_ids=True).generate(references=True)

vaccination_date_generator = FieldGenerator(
    field="occurrenceDateTime",
    generator_function=lambda: pendulum.now().to_date_string()
)

first_vax_params = GeneratorParameters(
    field_values=[
        FieldValue(field="vaccineCode", value=vaccination_code),
        FieldValue(field="status", value="completed"),
    ],
    field_generators=[
        vaccination_date_generator
    ]
)
vaccination_generator = ResourceGenerator("Immunization", generator_parameters=first_vax_params)
dataset_generator.add_resource(vaccination_generator, name="first_vaccination", likelihood=0.8)

dataset = dataset_generator.generate(ids=True)

dataset.upload(server)

Check if our server now has covid patients

In [ ]:
covid_query = server.query("Patient").has(
    resource="Condition",
    search_param="code",
    operator=QueryOperators.eq,
    value="RA01.0",
    reference_param="subject",
).include(
    resource="Condition",
    reference_param="subject",
    reverse=True
)

covid_response = covid_query.all()
covid_response

## Transferring resources from one server to another

Use the `transfer()` function on a server object to transfer resources from one server to another while keeping referential integrity and using server assigned ids.
The transfer is a three-step process:
1. Analyze the resources to be transferred and build a DAG modeling the references
2. Obtain any missing resources that are referenced from the source server
3. Upload the resources to the target server based on the reference DAG

In [ ]:
# define a new server
transfer_api_url = "https://demo.personalhealthtrain.de/demo-fhir-4"
transfer_server = FhirServer(api_address=transfer_api_url, username=username, password=password)

server.transfer(transfer_server, covid_response)

In [ ]:
transfer_covid_query = transfer_server.query("Patient").has(
    resource="Condition",
    search_param="code",
    operator=QueryOperators.eq,
    value="RA01.0",
    reference_param="subject",
)
transfer_covid_query.all()

In [ ]:
transfer_server.query("Condition").all()

In [35]:
transfer_server.query("Condition").all()

<QueryResponse(resource=Condition, n=171)>

<center><h2>Questions?</h2></center>
<center><h3>What feature would you like to have in a FHIR library?</h3></center>